In [ ]:
#loading the unprocessed data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
import re
import string
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn import model_selection, naive_bayes, svm
from sklearn.naive_bayes import GaussianNB
import os
nltk.download('vader_lexicon')
project_data = pd.read_csv('craiglist_1.2k_appliances_data.csv')

In [ ]:
project_data

In [ ]:
#droping not so relevant columns
project_data = project_data.drop(['link','Title','Price'], axis=1)
project_data

In [ ]:
#glancing at project_data dataframe
num_of_colmns_proj = len(project_data.columns)
num_of_rows_proj   = len(project_data)
print('Number of Columns in Project data   : ',num_of_colmns_proj)
print('Number of rows in Project data      : ',num_of_rows_proj)

In [ ]:
#defining the function to calculate the length of the string, number of words, and presence of digit in each text field
import regex as re
def check_len(summary):
  size = len(summary)
  return size

def check_word_count(summary):
  countt=0
  for word in summary.split(' '):
    countt+=1
  return countt

def digit_counter(summary):
  temp = bool(re.search('\d',summary))
  if temp:
    return 1
  else:
    return 0

In [ ]:
#introducing new columns to denote the length of project summary and number of words in it
project_data['summary_size']     = project_data.apply(lambda x: check_len(str(x['Description'])),axis=1)
project_data['words_in_summary'] = project_data.apply(lambda x: check_word_count(str(x['Description'])),axis=1)
project_data['digit_in_summary'] = project_data.apply(lambda x: digit_counter(str(x['Description'])),axis=1)
project_data.head(6)


In [ ]:
#understanding the variation in the length of summary and number of words in it
summ_max = project_data['summary_size'].max()
summ_min = project_data['summary_size'].min()
print('='*45)
print('Maximum lenth of summary is  : ',summ_max)
print('Minimum lenth of summary is  : ',summ_min)
print('='*45)

In [ ]:
#understanding the variation in number of words in summary
from collections import Counter
summ_word_counter    = Counter()
word_count_summ      = list(project_data['words_in_summary'])
summ_word_counter.update(word_count_summ)
summ_word_counter    = dict(summ_word_counter)
summ_word_counter    = dict(sorted(summ_word_counter.items(), key=lambda kv: kv[0]))
plt.plot(list(summ_word_counter.keys()),list(summ_word_counter.values()))
plt.scatter(list(summ_word_counter.keys()),list(summ_word_counter.values()))
plt.xlabel('Number of words in each summary')
plt.ylabel('Number of projects')
plt.grid()
plt.show()

In [ ]:
#variation in our class label
project_data['Condition'].value_counts()

In [ ]:
#ploting the box plot for the approved and rejected summaries
app_sum_word_count = project_data[project_data['Condition']=='excellent']['words_in_summary']
excellent_sum_word_count = app_sum_word_count.values
app_sum_word_count = project_data[project_data['Condition']=='new']['words_in_summary']
new_sum_word_count = app_sum_word_count.values
app_sum_word_count = project_data[project_data['Condition']=='good']['words_in_summary']
good_sum_word_count = app_sum_word_count.values
app_sum_word_count = project_data[project_data['Condition']=='like new']['words_in_summary']
like_new_sum_word_count = app_sum_word_count.values
plt.boxplot([excellent_sum_word_count, new_sum_word_count, good_sum_word_count,like_new_sum_word_count  ])
plt.xticks([1,2,3,4],('excellent_sum_word_count', 'new_sum_word_count', 'good_sum_word_count','like_new_sum_word_count'))
plt.ylabel('Words in project summaries')
plt.grid()
plt.show()

In [ ]:
#Preprocessing the Column : Description
import re
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
#creating a list of stop words
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
#using above functions and list of stop words to preprocess the textual data
from tqdm import tqdm
def preprocess_text(text_data):
    preprocessed_text = []
    for sentance in tqdm(text_data):
        sentance = str(sentance)
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [ ]:
#preprocessing the project title using above defined functions
cleaned_Description = preprocess_text(project_data['Description'].values)

In [ ]:
#creating a new column with preprocessed Description
project_data['cleaned_Description'] = np.array(cleaned_Description)
project_data

In [ ]:
Excellent_description = list(project_data[project_data['Condition'] == 'excellent']['Description'].values)
Excellent_description_str = ''
for doc in Excellent_description:
    Excellent_description_str += doc
print(len(Excellent_description_str))

new_description = list(project_data[project_data['Condition'] == 'new']['Description'].values)
new_description_str = ''
for doc in new_description:
    new_description_str += doc
print(len(new_description_str))

cat_1 = Excellent_description_str + ' ' + new_description_str

print(len(Excellent_description_str))
print(len(Excellent_description),len(new_description))
Excellent_description = list(project_data[project_data['Condition'] == 'excellent']['Description'].values)
Excellent_description = list(project_data[project_data['Condition'] == 'excellent']['Description'].values)
Excellent_description = list(project_data[project_data['Condition'] == 'excellent']['Description'].values)
Excellent_description_str = ''
for doc in Excellent_description:
    Excellent_description_str += doc
len(cat_1)

In [ ]:
#generating a word cloud using word cloud library
Word_box_pre = cat_1
from wordcloud import WordCloud, STOPWORDS
#STOPWORDS.add('new')
#STOPWORDS.add('box')
#STOPWORDS.add('will')
wordcloud = WordCloud(width = 800,height = 800,background_color ='white',min_font_size=10, stopwords=STOPWORDS).generate(Word_box_pre)
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
Good_description = list(project_data[project_data['Condition'] == 'good']['Description'].values)
Good_description_str = ''
for doc in Good_description:
    Good_description_str += doc
print(len(Good_description_str))

like_new_description = list(project_data[project_data['Condition'] == 'like new']['Description'].values)
like_new_description_str = ''
for doc in new_description:
    like_new_description_str += doc
print(len(like_new_description_str))

fair_description = list(project_data[project_data['Condition'] == 'fair']['Description'].values)
fair_description_str = ''
for doc in fair_description:
    fair_description_str += doc
print(len(fair_description_str))

cat_2 = Good_description_str + ' ' + like_new_description_str + ' ' + fair_description_str

In [ ]:
#generating a word cloud using word cloud library
Word_box_pre = cat_2
from wordcloud import WordCloud, STOPWORDS
STOPWORDS.add('new')
#STOPWORDS.add('box')
#STOPWORDS.add('will')
wordcloud = WordCloud(width = 800,height = 800,background_color ='white',min_font_size=10, stopwords=STOPWORDS).generate(Word_box_pre)
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
#Generating sentiment scores for the cleaned descriptions
sid = SentimentIntensityAnalyzer()
project_data['positive_sentiments'] = project_data.apply(lambda x: sid.polarity_scores(str(x['cleaned_Description']))['pos'],axis=1)
project_data


In [ ]:
#dropping rows with no decription
project_data = project_data[project_data['Description'].notna()]
nan_rows = project_data[project_data['Description'].isnull()]
nan_rows

In [ ]:
#adding new attributes to the data set
project_data['ex_flag'] = project_data['Description'].str.contains('excellent|Excellent').astype(int)
project_data['new_flag'] = project_data['Description'].str.contains('new|New').astype(int)
project_data['go_flag'] = project_data['Description'].str.contains('good|Good').astype(int)

In [ ]:
#publishing the dataframe 
project_data.head(3)

In [ ]:
For_test = project_data[project_data['Condition'].isnull()]
For_test.head(6)

In [ ]:
#variation in our class label
project_data['Condition'].value_counts()

In [ ]:
#mapping column values with digits
project_data['Condition'] = project_data['Condition'].map({'excellent': 0, 'new': 0, 'good':1, 'like new':1, 'fair':1 })


In [ ]:
#variation in our class label now
project_data['Condition'].value_counts()

In [ ]:
#dropping rows with Nan coondition
project_data.dropna(subset=['Condition'],inplace = True)

In [ ]:
project_data.head(6)

In [ ]:
#partioning columns as per the nature of metadata
y = project_data['Condition'].values
X = project_data.drop(['Description','Condition'], axis=1)

In [ ]:
#viewing data that finally need to be processed
X.head(3)

In [ ]:
#spliting the entire data into train and test, then further train data into train and cross validate
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
#getting data for our testing
y_test = For_test['Condition'].values
X_test = For_test.drop(['Description','Condition'], axis=1)
X_test.head(3)

In [ ]:
#creating a big list of all the features through train data
big_list_features = []

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['go_flag'].values.reshape(-1,1))
big_list_features.extend(['go_flag'])

In [ ]:
#using fitted Normalizer
X_train_go_flag_norm = normalizer.transform(X_train['go_flag'].values.reshape(-1,1))
X_cv_go_flag_norm = normalizer.transform(X_cv['go_flag'].values.reshape(-1,1))
X_test_go_flag_norm = normalizer.transform(X_test['go_flag'].values.reshape(-1,1))

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_go_flag_norm.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_train_go_flag_norm.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_train_go_flag_norm.shape,'Y test  shape :', y_test.shape)

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['new_flag'].values.reshape(-1,1))
big_list_features.extend(['new_flag'])

In [ ]:
#using fitted Normalizer
X_train_new_flag_norm = normalizer.transform(X_train['new_flag'].values.reshape(-1,1))
X_cv_new_flag_norm = normalizer.transform(X_cv['new_flag'].values.reshape(-1,1))
X_test_new_flag_norm = normalizer.transform(X_test['new_flag'].values.reshape(-1,1))

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_new_flag_norm.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_new_flag_norm.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_new_flag_norm.shape,'Y test  shape :', y_test.shape)

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['ex_flag'].values.reshape(-1,1))
big_list_features.extend(['ex_flag'])

In [ ]:
#using fitted Normalizer
X_train_ex_flag_norm = normalizer.transform(X_train['ex_flag'].values.reshape(-1,1))
X_cv_ex_flag_norm = normalizer.transform(X_cv['ex_flag'].values.reshape(-1,1))
X_test_ex_flag_norm = normalizer.transform(X_test['ex_flag'].values.reshape(-1,1))

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_ex_flag_norm.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_ex_flag_norm.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_ex_flag_norm.shape,'Y test  shape :', y_test.shape)

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['digit_in_summary'].values.reshape(-1,1))
big_list_features.extend(['digit_in_summary'])

In [ ]:
#using fitted Normalizer
X_train_digit_in_summary_norm = normalizer.transform(X_train['digit_in_summary'].values.reshape(-1,1))
X_cv_digit_in_summary_norm = normalizer.transform(X_cv['digit_in_summary'].values.reshape(-1,1))
X_test_digit_in_summary_norm = normalizer.transform(X_test['digit_in_summary'].values.reshape(-1,1))

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_digit_in_summary_norm.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_digit_in_summary_norm.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_digit_in_summary_norm.shape,'Y test  shape :', y_test.shape)

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['words_in_summary'].values.reshape(-1,1))
big_list_features.extend(['words_in_summary'])

In [ ]:
#using fitted Normalizer
X_train_words_in_summary = normalizer.transform(X_train['words_in_summary'].values.reshape(-1,1))
X_cv_words_in_summary = normalizer.transform(X_cv['words_in_summary'].values.reshape(-1,1))
X_test_words_in_summary = normalizer.transform(X_test['words_in_summary'].values.reshape(-1,1))

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_words_in_summary.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_words_in_summary.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_words_in_summary.shape,'Y test  shape :', y_test.shape)

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['summary_size'].values.reshape(-1,1))
big_list_features.extend(['summary_size'])

In [ ]:
#using fitted Normalizer
X_train_summary_size = normalizer.transform(X_train['summary_size'].values.reshape(-1,1))
X_cv_summary_size = normalizer.transform(X_cv['summary_size'].values.reshape(-1,1))
X_test_summary_size = normalizer.transform(X_test['summary_size'].values.reshape(-1,1))

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_summary_size.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_summary_size.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_summary_size.shape,'Y test  shape :', y_test.shape)

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['positive_sentiments'].values.reshape(-1,1))
big_list_features.extend(['positive_sentiments'])

In [ ]:
#using fitted Normalizer
X_train_positive_sentiments = normalizer.transform(X_train['positive_sentiments'].values.reshape(-1,1))
X_cv_positive_sentiments = normalizer.transform(X_cv['positive_sentiments'].values.reshape(-1,1))
X_test_positive_sentiments = normalizer.transform(X_test['positive_sentiments'].values.reshape(-1,1))

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_positive_sentiments.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_positive_sentiments.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_positive_sentiments.shape,'Y test  shape :', y_test.shape)

In [ ]:
#fit vectorizing the column of train data using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1,ngram_range=(1,1), max_features=5000)
vectorizer.fit(X_train['cleaned_Description'].values)
print('Features on one hot encoding for the column :',len(vectorizer.get_feature_names()))
big_list_features.extend(vectorizer.get_feature_names())

In [ ]:
#using fitted count vectorizer to convert text to vector
X_train_cleaned_cleaned_Description = vectorizer.transform(X_train['cleaned_Description'].values)
X_cv_cleaned_cleaned_Description = vectorizer.transform(X_cv['cleaned_Description'].values)
X_test_cleaned_cleaned_Description = vectorizer.transform(X_test['cleaned_Description'].values)

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_cleaned_cleaned_Description.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_cleaned_cleaned_Description.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_cleaned_cleaned_Description.shape,'Y test  shape :', y_test.shape)

### Only run below cell if want to use TF-idf

In [ ]:
#fit vectorizing the column of train data using TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=4,ngram_range=(1,4), max_features=5000)
vectorizer.fit(X_train['cleaned_Description'].values)
print('TFIDF Vocabulary for the column :',len(vectorizer.vocabulary_))

In [ ]:
#using fitted count vectorizer to convert text to vector
X_train_cleaned_cleaned_Description = vectorizer.transform(X_train['cleaned_Description'].values)
X_cv_cleaned_cleaned_Description = vectorizer.transform(X_cv['cleaned_Description'].values)
X_test_cleaned_cleaned_Description = vectorizer.transform(X_test['cleaned_Description'].values)

In [ ]:
#printing the shape of vectors obtained post transformation and validating the number of rows using 'y'
print('Train vector shape : ',X_train_cleaned_cleaned_Description.shape,'Y train shape :',y_train.shape)
print('C.V.  vector shape : ',X_cv_cleaned_cleaned_Description.shape,'Y C.V.  shape :', y_cv.shape)
print('Test  vector shape : ',X_test_cleaned_cleaned_Description.shape,'Y test  shape :', y_test.shape)

### Trying different combinations of our variable

In [ ]:
#concatinating features using hstack from scipy
from scipy.sparse import hstack
X_tr_bow = hstack((X_train_ex_flag_norm, X_train_digit_in_summary_norm, X_train_cleaned_cleaned_Description,9*X_train_positive_sentiments)).tocsr()
X_cr_bow = hstack((X_cv_ex_flag_norm, X_cv_digit_in_summary_norm, X_cv_cleaned_cleaned_Description,9*X_cv_positive_sentiments)).tocsr()
X_te_bow = hstack((X_test_ex_flag_norm, X_test_digit_in_summary_norm, X_test_cleaned_cleaned_Description, 9*X_test_positive_sentiments)).tocsr()


In [ ]:
#concatinating features using hstack from scipy
from scipy.sparse import hstack
X_tr_bow = hstack((X_train_digit_in_summary_norm, X_train_cleaned_cleaned_Description,X_train_positive_sentiments)).tocsr()
X_cr_bow = hstack((X_cv_digit_in_summary_norm, X_cv_cleaned_cleaned_Description,X_cv_positive_sentiments)).tocsr()
X_te_bow = hstack((X_test_digit_in_summary_norm, X_test_cleaned_cleaned_Description, X_test_positive_sentiments)).tocsr()


In [ ]:
#base case
from scipy.sparse import hstack
X_tr_bow = X_train_cleaned_cleaned_Description
X_cr_bow = X_cv_cleaned_cleaned_Description
X_te_bow = X_test_cleaned_cleaned_Description

In [ ]:
#concatinating features using hstack from scipy
from scipy.sparse import hstack
X_tr_bow = hstack((9*X_train_ex_flag_norm, 3*X_train_new_flag_norm, X_train_go_flag_norm,X_train_digit_in_summary_norm,X_train_words_in_summary,X_train_summary_size,X_train_cleaned_cleaned_Description,X_train_positive_sentiments)).tocsr()
X_cr_bow = hstack((9*X_cv_ex_flag_norm, 3*X_cv_new_flag_norm, X_cv_go_flag_norm,X_cv_digit_in_summary_norm,X_cv_words_in_summary,X_cv_summary_size,X_cv_cleaned_cleaned_Description,X_cv_positive_sentiments)).tocsr()
X_te_bow = hstack((9*X_test_ex_flag_norm, 3*X_test_new_flag_norm, X_test_go_flag_norm,X_test_digit_in_summary_norm,X_test_words_in_summary,X_test_summary_size,X_test_cleaned_cleaned_Description, X_test_positive_sentiments)).tocsr()


In [ ]:
#publishing the shape of final features
print(X_tr_bow.shape, y_train.shape)
print(X_cr_bow.shape, y_cv.shape)
print(X_te_bow.shape, y_test.shape)

In [ ]:
#confirming with the dimension of our list of features
len(big_list_features)

In [ ]:
#below is the base case, with only one hot encoding and ridge classifier
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

train_acc  = []
cv_acc     = []
train_val  = []
cv_val     = []
log_i      = []
K = [10**-10,10**-6,10**-2,10**2, 10**6, 10**10]

for i in tqdm(K):
    Classifier = RidgeClassifier(alpha= i)
    Classifier.fit(X_tr_bow,y_train)
    y_train_pred = Classifier.decision_function(X_tr_bow)
    pred_probs = np.exp(y_train_pred)
    pred_probs = np.where(pred_probs>0.90, 1, 0)
    accuracy_val = accuracy_score(y_train, pred_probs)
    train_acc.append(accuracy_val)
    y_train_pred = Classifier.decision_function(X_cr_bow)
    pred_probs = np.exp(y_train_pred)
    pred_probs = np.where(pred_probs>0.90, 1, 0)
    accuracy_val = accuracy_score(y_cv, pred_probs)
    cv_acc.append(accuracy_val) 
    
for i in K:
    log_i.append(math.log10(i))
plt.plot(log_i, train_acc, label='Train ACC')
plt.plot(log_i, cv_acc, label='CV ACC')
plt.scatter(log_i, train_acc, label='Train ACC points')
plt.scatter(log_i, cv_acc, label='CV ACC points')
plt.legend()
plt.xlabel("Alpha: hyperparameter")
plt.ylabel("ACC")
plt.title("Accuracy PLOTS")
plt.grid()
plt.show()

In [ ]:
#applying the Bernoulli Naive Bayes on the prepared data set
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

train_acc  = []
cv_acc     = []
train_val  = []
cv_val     = []
log_i      = []
#K = [10**-7,10**-6,10**-5,10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1,10**2,10**3,10**4]

for i in tqdm(K):
    NB_Classifier = MultinomialNB(alpha = i)
    NB_Classifier.fit(X_tr_bow,y_train)
    y_train_pred     = NB_Classifier.predict(X_tr_bow)
    y_cv_pred        = NB_Classifier.predict(X_cr_bow)
    train_acc.append(accuracy_score(y_train,y_train_pred))
    cv_acc.append(accuracy_score(y_cv, y_cv_pred))
for i in K:
    log_i.append(math.log10(i))
plt.plot(log_i, train_acc, label='Train ACC')
plt.plot(log_i, cv_acc, label='CV ACC')
plt.scatter(log_i, train_acc, label='Train ACC points')
plt.scatter(log_i, cv_acc, label='CV ACC points')
plt.legend()
plt.xlabel("Alpha: hyperparameter")
plt.ylabel("ACC")
plt.title("Accuracy PLOTS")
plt.grid()
plt.show()

In [ ]:
#applying the Random Forest Classifier on the prepared data set
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

train_acc  = []
cv_acc     = []
depth      = []
K = [2,4,6,8,10,12,15,20,27,30,33,36]

for i in tqdm(K):
    Rf_clf = RandomForestClassifier(max_depth=i, random_state=0)
    Rf_clf.fit(X_tr_bow,y_train)
    y_train_pred     = Rf_clf.predict(X_tr_bow)
    y_cv_pred        = Rf_clf.predict(X_cr_bow)
    train_acc.append(accuracy_score(y_train,y_train_pred))
    cv_acc.append(accuracy_score(y_cv, y_cv_pred))
for i in K:
    depth.append((i))
plt.plot(depth, train_acc, label='Train ACC')
plt.plot(depth, cv_acc, label='CV ACC')
plt.scatter(depth, train_acc, label='Train ACC points')
plt.scatter(depth, cv_acc, label='CV ACC points')
plt.legend()
plt.xlabel("Depth: hyperparameter")
plt.ylabel("ACC")
plt.title("Accuracy PLOTS")
plt.grid()
plt.show()

In [ ]:
print('The max accuracy we are getting is',cv_acc[-2],'with depth of',K[-2],'are parameter for random forest')

In [ ]:
#mapping column values with digits
y_train_pred     = Rf_clf.predict(X_te_bow)
For_test['Condition'] = (y_train_pred)
For_test = For_test.astype({'Condition':'int'})
For_test['Condition'] = For_test['Condition'].map({ 0 : 'excellent',  1: 'good' })
For_test.head(3)

In [ ]:
excellent_description = list(For_test[For_test['Condition'] == 'excellent']['Description'].values)
for doc in excellent_description[15:20]:
    print(doc)
#print(excellent_description[12])
#print(excellent_description[21])

In [ ]:
good_description = list(For_test[For_test['Condition'] == 'good']['Description'].values)
print(good_description[8])
print(good_description[16])

### --- End of Notebook ---